<a href="https://colab.research.google.com/github/bazsi04/Szakdolgozat/blob/main/Szakdoga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Nem-negatív élsúlyozás, szig. mon. csökk. legrövidebb út

In [ ]:
from heapq import heappush, heappop

def nem_neg_mon(input):
    V, adj_list, s = input
    P = []
    S = {s}
    p_v = {s: s}
    p_e = {}
    R = {s: 0}
    c = {}

    for u in adj_list:
        for (v, weight) in adj_list[u]:
            c[u,v] = weight

    h = {u: 0 for u in V}
    numedge = {u: len(adj_list[u]) for u in V}
    F = {u: sorted(adj_list[u], key=lambda x: x[1]) for u in V}

    for v, K in adj_list[s]:
        heappush(P, (K, s, v))
        p_e[s,v] = s
        if v not in R or K_total < R[t]:
            R[v] = K
            p_v[v] = (s,v)


    while P:
        K_uv, u, v = heappop(P)
        if h[v] > numedge[v]:
            continue


        for i in range(h[v], numedge[v]):
            t, K_vt = F[v][i]
            if c[v,t] >= c[u,v]:
                break

            K_total = K_uv + c[v,t]
            p_e[v,t] = (u, v)
            if t not in R or K_total < R[t]:
                R[t] = K_total
                p_v[t] = (v,t)
                heappush(P, (K_total, v, t))

            h[v] += 1

    return p_v, p_e, R


### Konzervatív élsúlyozás, szig. mon. csökk. legrövidebb út

In [ ]:
from heapq import heappush, heappop

def konz_mon(input):
    V, adj_list, s = input
    P = []
    S = {s}
    p_v = {s: s}
    p_e = {}
    R = {s: 0}
    c = {}

    for u in adj_list:
        for (v, weight) in adj_list[u]:
            c[u, v] = weight

    h = {u: 0 for u in V}
    numedge = {u: len(adj_list[u]) for u in V}
    F = {u: sorted(adj_list[u], key=lambda x: x[1]) for u in V}

    for v, cost in adj_list[s]:
        heappush(P, (cost, s, v))
        p_e[s, v] = s
        if v not in R or cost < R[v]:
            R[v] = cost
            p_v[v] = (s, v)

    while P:
        K_uv, u, v = heappop(P)
        if h[v] >= numedge[v]:
            continue

        for i in range(h[v], numedge[v]):
            t, c_vt = F[v][i]
            if c_vt >= c[u, v]:
                break

            K_total = K_uv + c_vt
            p_e[v, t] = (u, v)
            if t not in R or K_total < R[t]:
                R[t] = K_total
                p_v[t] = (v, t)
                heappush(P, (K_total, v, t))

            h[v] += 1

    return p_v, p_e, R


### Nem-negatív csúcssúlyozás minimális út

In [ ]:
def csucs_min(input):
    V, adj_list, s = input
    P = []
    S = set()
    M = set(V) - {s}
    p_v = {s: s}
    K = {s: 0}
    w = {}

    for u in adj_list:
        for v, weight in adj_list[u]:
            w[v] = weight

    heappush(P, (w.get(s, 0), s))
    while P:
        K_u, u = heappop(P)
        if u in S:
            continue
        S.add(u)

        for v, _ in adj_list.get(u, []):
            if v in M:
                K[v] = K[u] + w.get(v, 0)
                p_v[v] = p_v[u]
                heappush(P, (K[v], v))
                M.remove(v)

    return p_v, K

### Legfeljebb k élű minimális út

In [ ]:
def k_el_min(input):
    V, adj_list, s, k = input
    K = {}
    p = {}
    c = {}

    for u in adj_list:
        for (v, weight) in adj_list[u]:
            c[(u, v)] = weight


    for u in V:
        K[(0, u)] = float('inf')
    K[(0, s)] = 0
    p[(0, s)] = s

    for j in range(1, k + 1):
        for u in V:
            K[(j, u)] = float('inf')
            p[(j, u)] = p.get((j - 1, u), None)

        for u in adj_list:
            for v, cost in adj_list[u]:
                prev = K.get((j - 1, u), float('inf'))
                if K[(j, v)] > prev + cost:
                    K[(j, v)] = prev + cost
                    p[(j, v)] = u

    return K, p

### k-hosszú út

In [ ]:
def coloring(V, adj_list, k, seed = None):
    if seed is not None:
        random.seed(seed)

    node_colors = {v: random.randint(0, k-1) for v in V}


    edge_colors = {}
    for u in adj_list:
        for v, _ in adj_list[u]:
            edge_colors[(u, v)] = node_colors[v]

    return node_colors, edge_colors

In [ ]:
from itertools import combinations
import random

def colorcoding(input, seed = None):
    V, adj_list, s, k = input
    from collections import defaultdict
    f = {}
    p = {}
    colors = set(range(0,k))
    _, edge_colors = coloring(V, adj_list, k, seed)
    #print("colors: ",edge_colors)
    w = {}
    for u in adj_list:
        for (v, weight) in adj_list[u]:
            w[u, v] = weight

    E = [(u, v) for u in adj_list for (v,_) in adj_list[u]]

    rev_adj_list = {}
    # Színek részhalmazai elemszám szerint növekvő sorrendben
    P = [frozenset(comb) for i in range(1, k+1) for comb in combinations(colors, i)]


    for p_set in P:
        for (u, v) in E:
            f[p_set, (u, v)] = False


            if u == s and edge_colors[(u, v)] in p_set:
                f[p_set, (u, v)] = True
                p[p_set, (u, v)] = s


    for p_set in P[k:]:
        for (u, v) in E:
            f[p_set, (u, v)] = False
            if edge_colors[(u, v)] in p_set:
                for (t, uu) in E:
                    if uu == u and f.get((p_set - {edge_colors[(u, v)]}, (t, uu)), True) and w[(t, uu)] < w[(u, v)] and v != s:
                        f[p_set, (u, v)] = True
                        p[p_set, (u, v)] = (t, u)

    for p_set in P[k:]:
        for (u, v) in E:
            f[p_set, (u, v)] = False
            if edge_colors[(u, v)] in p_set:
                for (t, uu) in E:
                    if uu == u and f.get((p_set - {edge_colors[(u, v)]}, (t, uu)), True) and w[(t, uu)] < w[(u, v)] and v != s:
                        f[p_set, (u, v)] = True
                        p[p_set, (u, v)] = (t, u)


    f_v = {}
    p_v = {}
    for (u, v) in E:
        if f.get((P[-1], (u, v)), True):
            f_v[v] = True
            p_v[v] = (u, v)

    return f_v, p_v, f, p

In [ ]:
from itertools import combinations
import random

def colorcoding_fast(input, seed = None):
    V, adj_list, s, k = input
    from collections import defaultdict
    f = {}
    p = {}
    colors = set(range(0,k))
    _, edge_colors = coloring(V, adj_list, k, seed)
    #print("colors: ",edge_colors)
    """
    w = {}
    for u in adj_list:
        for (v, weight) in adj_list[u]:
            w[u, v] = weight

    E = [(u, v) for u in adj_list for (v,_) in adj_list[u]]
    """
    rev_lst = {u: [] for u in adj_list}

    for u in adj_list:
        for v, weight in adj_list[u]:
            rev_lst[v].append((u, weight))

    # Színek részhalmazai elemszám szerint növekvő sorrendben
    P = [frozenset(comb) for i in range(1, k+1) for comb in combinations(colors, i)]


    for p_set in P:
        for u in adj_list:
            for v, _ in adj_list[u]:
                f[p_set, (u, v)] = False


            if u == s and edge_colors[(u, v)] in p_set:
                f[p_set, (u, v)] = True
                p[p_set, (u, v)] = s

    """
    for p_set in P[k:]:
        for (u, v) in E:
            f[p_set, (u, v)] = False
            if edge_colors[(u, v)] in p_set:
                for (t, uu) in E:
                    if uu == u and f.get((p_set - {edge_colors[(u, v)]}, (t, uu)), True) and w[(t, uu)] < w[(u, v)] and v != s:
                        f[p_set, (u, v)] = True
                        p[p_set, (u, v)] = (t, u)
    """
    for p_set in P[k:]:
        for u in adj_list:
            for (v, w) in adj_list[u]:
                f[p_set, (u, v)] = False
                if edge_colors[(u, v)] in p_set:
                    for (t, ww) in rev_lst[u]:
                        if f.get((p_set - {edge_colors[(u, v)]}, (t, u)), True) and ww < w and v != s:
                            f[p_set, (u, v)] = True
                            p[p_set, (u, v)] = (t, u)


    f_v = {}
    p_v = {}
    """
    for (u, v) in E:
        if f.get((P[-1], (u, v)), True):
            f_v[v] = True
            p_v[v] = (u, v)
    """
    for u in adj_list:
        for v, _ in adj_list[u]:
            if f.get((P[-1], (u, v)), True):
                f_v[v] = True
                p_v[v] = (u, v)

    return f_v, p_v, f, p

In [ ]:
V = [1, 2, 3, 4]
adj_list = {
    1: [(2, 1.0), (3, 2.0)],
    2: [(3, 3.0)],
    3: [(4, 4.5)],
    4: []
}
s = 1
k = 3  # k színből álló út keresése

# Bemenet összeállítása és futtatás
input = (V, adj_list, s, k)
f_v, p_v, f, p = colorcoding(input)
print("f_v",f_v)
print("p_v",p_v)
print("f",f)
print("p",p)

f_v {}
p_v {}
f {(frozenset({0}), (1, 2)): False, (frozenset({0}), (1, 3)): True, (frozenset({0}), (2, 3)): False, (frozenset({0}), (3, 4)): False, (frozenset({1}), (1, 2)): False, (frozenset({1}), (1, 3)): False, (frozenset({1}), (2, 3)): False, (frozenset({1}), (3, 4)): False, (frozenset({2}), (1, 2)): True, (frozenset({2}), (1, 3)): False, (frozenset({2}), (2, 3)): False, (frozenset({2}), (3, 4)): False, (frozenset({0, 1}), (1, 2)): False, (frozenset({0, 1}), (1, 3)): False, (frozenset({0, 1}), (2, 3)): False, (frozenset({0, 1}), (3, 4)): False, (frozenset({0, 2}), (1, 2)): False, (frozenset({0, 2}), (1, 3)): False, (frozenset({0, 2}), (2, 3)): True, (frozenset({0, 2}), (3, 4)): True, (frozenset({1, 2}), (1, 2)): False, (frozenset({1, 2}), (1, 3)): False, (frozenset({1, 2}), (2, 3)): False, (frozenset({1, 2}), (3, 4)): False, (frozenset({0, 1, 2}), (1, 2)): False, (frozenset({0, 1, 2}), (1, 3)): False, (frozenset({0, 1, 2}), (2, 3)): False, (frozenset({0, 1, 2}), (3, 4)): False}
p {

### Gráf generálás

In [ ]:
import random

def erdos_renyi(n, p, min_w = 0, max_w = 1000, directed=True):
    V = list(range(1, n+1))
    Edges = {v: [] for v in V}

    for u in V:
        for v in V:
            if u == v:
                continue
            if random.random() < p:
                weight = random.uniform(min_w, max_w)
                Edges[u].append((v, weight))
                if not directed:
                    Edges[v].append((u, weight))

    return V, Edges, 1

In [ ]:
class TreeNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None

def generate_binary_tree(n, min_w = 0, max_w = 1000):
    if n == 0:
        return None, {}

    root = TreeNode(1)
    nodes = [root]
    edges = {1: []}


    weight = max_w

    for i in range(1, n):
        parent = nodes[(i-1) // 2]

        if parent.left is None:
            child = TreeNode(i+1)
            parent.left = child
            weight = random.uniform(min_w, weight)
            edges[parent.val].append((child.val, weight))
            edges[child.val] = []
            nodes.append(child)
        elif parent.right is None:
            child = TreeNode(i+1)
            parent.right = child
            weight = random.uniform(min_w, weight)
            edges[parent.val].append((child.val, weight))
            edges[child.val] = []
            nodes.append(child)
        else:
            break

    return list(range(1, n+1)), edges, 1

In [ ]:
def complete_graph(n, min_w = 0, max_w = 10):
    V = list(range(1, n+1))
    edges = {u: [(v, random.uniform(min_w, max_w)) for v in V if v != u] for u in V}
    return V, edges, 1

### Tesztelés

In [ ]:
"""
V = [1, 2, 3, 4]
adj_list = {
    1: [(2, 10), (3, 2)],
    2: [(4, 2.1)],
    3: [(2, 1)],
    4: []
}
s = 1
"""
V, adj_list, s = erdos_renyi(20, 0.2)
#V, adj_list, s = generate_binary_tree(2047)
#V, adj_list, s = complete_graph(5)

input = (V, adj_list, s)

print(adj_list)

pv, pe, R = nem_neg_mon(input)

print("Parent Vertex Pointers:", pv)
print("Parent Edge Pointers:", pe)
print("Shortest Path Costs:", R)
print("Reached:", len(R))

{1: [(8, 920.7664379818925), (15, 103.29705088653573)], 2: [(3, 635.6496552383554), (9, 937.4657430402938), (17, 84.8149800027812), (18, 372.73919706691527), (20, 104.96418895027537)], 3: [(5, 557.9939017496323), (13, 222.43711915927733), (18, 209.77107548454742)], 4: [(7, 920.2604276237084), (13, 637.8402312279505), (15, 248.24313972477285), (17, 471.17957083784745)], 5: [(3, 133.9088745791954), (9, 802.6962606003535), (15, 78.95754469981765)], 6: [(2, 973.9983237829591), (7, 330.8314902050068), (12, 715.1331579959518), (14, 417.8023219455561)], 7: [(6, 359.9424723402712), (10, 670.2155423512661), (14, 237.94306495814575), (17, 236.97862921685197)], 8: [(5, 303.4342409987935), (6, 958.6557538737269), (12, 454.38688935306595), (19, 555.529313890589), (20, 507.09848703335615)], 9: [(4, 841.4170571934459), (15, 585.2474138575768), (16, 723.8137228077054), (19, 544.4816245238804)], 10: [(7, 777.3795982713981), (13, 516.4810037218771), (14, 959.8643817372104), (17, 710.5175225859982), (19,

In [ ]:
"""
V = [1, 2, 3, 4]
adj_list = {
    1: [(2, -3), (3, 1)],
    2: [(4, 3)],
    3: [(2, 2)],
    4: []
}
s = 1
"""
#V, adj_list, s = erdos_renyi(30, 0.2)
#V, adj_list, s = generate_binary_tree(2047)
V, adj_list, s = complete_graph(12)

print("adj: ", adj_list)


k = 1
i = 0
while i < 100:
    input = (V, adj_list, s, k)
    f, p, f_v, p_v = colorcoding_fast(input)

    for u in V:
        if f:
            print("f: ", f, "p: ", p)
            k += 1
            i = 0
            break
    i += 1

print(k)




adj:  {1: [(2, 6.668911631649), (3, 9.679067578897257), (4, 4.475235013203694), (5, 8.57042134785585), (6, 8.252253886505773), (7, 1.505521133136244), (8, 4.643062924134666), (9, 6.277354997521468), (10, 6.103085336256791), (11, 3.9575505399597875), (12, 4.078129645412293)], 2: [(1, 0.6166551233801709), (3, 1.1280988705860684), (4, 0.7653149684843086), (5, 0.1967906938741093), (6, 9.456551451930874), (7, 7.606134728233693), (8, 5.611031987056418), (9, 0.8003680418699732), (10, 1.0172393670669067), (11, 4.548373006507525), (12, 2.879848013671489)], 3: [(1, 4.267724476784488), (2, 3.367153423832959), (4, 7.871145738349964), (5, 9.211800739820923), (6, 9.717396268410187), (7, 9.003000720962888), (8, 9.884385348697943), (9, 7.193069062311723), (10, 9.265149527892952), (11, 3.5079091758621663), (12, 5.034199965246177)], 4: [(1, 1.0152210249169435), (2, 5.536851360621324), (3, 8.542680287637511), (5, 9.353584247853956), (6, 3.7970225217908213), (7, 0.27870971710286074), (8, 2.923140157717662

In [ ]:
"""
V = [1, 2, 3, 4]
adj_list = {
    1: [(2, 10), (3, 2)],
    2: [(4, 2)],
    3: [(2, -1)],
    4: []
}
s = 1
"""
V, adj_list, s = erdos_renyi(2000, 0.02, min_w = -1000)


In [ ]:
def test(ex_out1, input1):
    assert ex_out1 == nem_neg_mon(input1)
    #assert ex_out2 == nem_neg_mon(input2)
    #assert ex_out3 == nem_neg_mon(input3)

ex_out1 = {1: 1, 2: (3, 2), 3: (1, 3), 4: (2, 4)}, {(1, 2): 1, (1, 3): 1, (3, 2): (1, 3), (2, 4): (1, 2)}, {1: 0, 2: 3, 3: 2, 4: 19}
input1 = [1, 2, 3, 4], {1: [(2, 10), (3, 2)], 2: [(4, 9)], 3: [(2, 1)], 4: []}, 1




test(ex_out1, input1)